In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model, Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import regularizers

dataset_path = "/content/drive/MyDrive/Music Dataset"

In [ ]:
image_size = (512, 512)
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

Found 5848 images belonging to 10 classes.
Found 1457 images belonging to 10 classes.


In [ ]:
# Define the .h5 file path
h5_file_path = "/content/drive/MyDrive/H5_files/model_5_40(70.6, 2.0).h5"

# Check if the .h5 file exists
if os.path.exists(h5_file_path):
    print("Loading existing model...")
    model = load_model(h5_file_path)
    initial_epoch = 40  # Start from the 21st epoch
    # Re-compile the model with a fresh optimizer
    sgd = SGD(learning_rate=0.002, momentum=0.9, nesterov=True, decay=0.0001)  # Create a new SGD instance
    model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy']) # Compile with new optimizer
else:
    print("No pre-trained model found. Building a new model...")
    initial_epoch = 0

    model = Sequential()

    model.add(Conv2D(16, (3, 3), strides=1, padding='same', activation='relu', input_shape=(image_size[0], image_size[1], 3)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, (3, 3), strides=1, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(256, (3, 3), strides=1, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(256, (3, 3), strides=1, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(512, (3, 3), strides=1, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dropout(0.3))
    model.add(Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.001)))
    model.add(Dropout(0.5))
    model.add(Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.001)))
    model.add(Dropout(0.5))
    model.add(Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.001)))
    model.add(Dropout(0.3))
    model.add(Dense(train_generator.num_classes, activation='softmax'))

    sgd = SGD(learning_rate=0.002, momentum=0.9, nesterov=True, decay=0.0001)
    model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()

# Initialize empty history dictionary to store losses
history_dict = {
    'loss': [],
    'val_loss': []
}

epochs = 100
for epoch in range(initial_epoch, epochs):
    print(f"Epoch {epoch + 1}/{epochs}")

    # Train the model for one epoch and capture history
    epoch_history = model.fit(
        train_generator,
        validation_data=validation_generator,
        epochs=1,  # Train for one epoch at a time
        verbose=1
    )

    # Store the loss and validation loss
    history_dict['loss'].append(epoch_history.history['loss'][0])
    history_dict['val_loss'].append(epoch_history.history['val_loss'][0])

    # Save the model every 10 epochs
    if (epoch + 1) % 5 == 0:
        save_path = f'/content/drive/MyDrive/H5_files/model_5_{epoch + 1:02d}.h5'
        model.save(save_path)
        print(f"Model saved at epoch {epoch + 1} to {save_path}")


Loading existing model...


/usr/local/lib/python3.10/dist-packages/keras/src/optimizers/base_optimizer.py:33: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_5 (Conv2D)                    │ (None, 512, 512, 16)        │             448 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_5                │ (None, 512, 512, 16)        │              64 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_5 (MaxPooling2D)       │ (None, 256, 256, 16)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_6 (Conv2D)                    │ (None, 256, 256, 64)        │           9,280 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_6                │ (None, 256, 256, 64)        │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_6 (MaxPooling2D)       │ (None, 128, 128, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_7 (Conv2D)                    │ (None, 128, 128, 256)       │         147,712 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_7                │ (None, 128, 128, 256)       │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_7 (MaxPooling2D)       │ (None, 64, 64, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_8 (Conv2D)                    │ (None, 64, 64, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_8                │ (None, 64, 64, 256)         │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_8 (MaxPooling2D)       │ (None, 32, 32, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_9 (Conv2D)                    │ (None, 32, 32, 512)         │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_9                │ (None, 32, 32, 512)         │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_9 (MaxPooling2D)       │ (None, 16, 16, 512)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 131072)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 131072)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 512)                 │      67,109,376 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 512)                 │              

 Total params: 69,189,898 (263.94 MB)

...

Model saved at epoch 70 to /content/drive/MyDrive/H5_files/model_5_70.h5
Epoch 71/100
183/183 ━━━━━━━━━━━━━━━━━━━━ 191s 1s/step - accuracy: 0.9707 - loss: 0.8103 - val_accuracy: 0.7426 - val_loss: 1.9247
Epoch 72/100
183/183 ━━━━━━━━━━━━━━━━━━━━ 185s 981ms/step - accuracy: 0.9728 - loss: 0.8054 - val_accuracy: 0.7447 - val_loss: 1.7747
Epoch 73/100
183/183 ━━━━━━━━━━━━━━━━━━━━ 187s 993ms/step - accuracy: 0.9706 - loss: 0.7962 - val_accuracy: 0.7474 - val_loss: 1.9139
Epoch 74/100
183/183 ━━━━━━━━━━━━━━━━━━━━ 191s 1s/step - accuracy: 0.9682 - loss: 0.8253 - val_accuracy: 0.7509 - val_loss: 1.8821
Epoch 75/100
183/183 ━━━━━━━━━━━━━━━━━━━━ 185s 984ms/step - accuracy: 0.9715 - loss: 0.7878 - val_accuracy: 0.7591 - val_loss: 1.8673


Model saved at epoch 75 to /content/drive/MyDrive/H5_files/model_5_75.h5
Epoch 76/100
183/183 ━━━━━━━━━━━━━━━━━━━━ 189s 1s/step - accuracy: 0.9767 - loss: 0.7595 - val_accuracy: 0.7289 - val_loss: 2.1656
Epoch 77/100
183/183 ━━━━━━━━━━━━━━━━━━━━ 183s 974ms/step - accuracy: 0.9780 - loss: 0.7604 - val_accuracy: 0.7666 - val_loss: 1.9293
Epoch 78/100
183/183 ━━━━━━━━━━━━━━━━━━━━ 188s 997ms/step - accuracy: 0.9767 - loss: 0.7370 - val_accuracy: 0.7076 - val_loss: 2.2402
Epoch 79/100
183/183 ━━━━━━━━━━━━━━━━━━━━ 182s 967ms/step - accuracy: 0.9770 - loss: 0.7390 - val_accuracy: 0.7392 - val_loss: 2.0509
Epoch 80/100
183/183 ━━━━━━━━━━━━━━━━━━━━ 182s 968ms/step - accuracy: 0.9808 - loss: 0.7191 - val_accuracy: 0.7179 - val_loss: 2.1219


Model saved at epoch 80 to /content/drive/MyDrive/H5_files/model_5_80.h5
Epoch 81/100
183/183 ━━━━━━━━━━━━━━━━━━━━ 188s 999ms/step - accuracy: 0.9852 - loss: 0.7022 - val_accuracy: 0.7385 - val_loss: 1.9946
Epoch 82/100
183/183 ━━━━━━━━━━━━━━━━━━━━ 193s 1s/step - accuracy: 0.9813 - loss: 0.7054 - val_accuracy: 0.7440 - val_loss: 2.0691
Epoch 83/100
183/183 ━━━━━━━━━━━━━━━━━━━━ 180s 957ms/step - accuracy: 0.9826 - loss: 0.6867 - val_accuracy: 0.7062 - val_loss: 2.1543
Epoch 84/100
183/183 ━━━━━━━━━━━━━━━━━━━━ 185s 978ms/step - accuracy: 0.9795 - loss: 0.6933 - val_accuracy: 0.7440 - val_loss: 1.9897
Epoch 85/100
183/183 ━━━━━━━━━━━━━━━━━━━━ 183s 968ms/step - accuracy: 0.9816 - loss: 0.6768 - val_accuracy: 0.7426 - val_loss: 1.9360


Model saved at epoch 85 to /content/drive/MyDrive/H5_files/model_5_85.h5
Epoch 86/100
183/183 ━━━━━━━━━━━━━━━━━━━━ 187s 993ms/step - accuracy: 0.9813 - loss: 0.6817 - val_accuracy: 0.7708 - val_loss: 1.8283
Epoch 87/100
183/183 ━━━━━━━━━━━━━━━━━━━━ 189s 1s/step - accuracy: 0.9783 - loss: 0.6737 - val_accuracy: 0.6651 - val_loss: 2.3743
Epoch 88/100
183/183 ━━━━━━━━━━━━━━━━━━━━ 182s 963ms/step - accuracy: 0.9819 - loss: 0.6537 - val_accuracy: 0.7522 - val_loss: 2.1693
Epoch 89/100
183/183 ━━━━━━━━━━━━━━━━━━━━ 183s 972ms/step - accuracy: 0.9808 - loss: 0.6498 - val_accuracy: 0.7708 - val_loss: 1.8548
Epoch 90/100
183/183 ━━━━━━━━━━━━━━━━━━━━ 184s 977ms/step - accuracy: 0.9868 - loss: 0.6189 - val_accuracy: 0.7680 - val_loss: 1.7024


Model saved at epoch 90 to /content/drive/MyDrive/H5_files/model_5_90.h5
Epoch 91/100
183/183 ━━━━━━━━━━━━━━━━━━━━ 183s 974ms/step - accuracy: 0.9814 - loss: 0.6463 - val_accuracy: 0.7563 - val_loss: 1.7608
Epoch 92/100
183/183 ━━━━━━━━━━━━━━━━━━━━ 184s 981ms/step - accuracy: 0.9895 - loss: 0.6095 - val_accuracy: 0.7598 - val_loss: 1.8579
Epoch 93/100
183/183 ━━━━━━━━━━━━━━━━━━━━ 194s 1s/step - accuracy: 0.9879 - loss: 0.6004 - val_accuracy: 0.7481 - val_loss: 2.1917
Epoch 94/100
 40/183 ━━━━━━━━━━━━━━━━━━━━ 1:52 789ms/step - accuracy: 0.9822 - loss: 0.6418

In [ ]:
def plot_loss(history_dict):
    plt.figure(figsize=(10, 5))
    plt.plot(history_dict['loss'], label='Training Loss')
    plt.plot(history_dict['val_loss'], label='Validation Loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

# Plot the loss graph after all epochs are completed
plot_loss(history_dict)